![Hand with calculator](calculator.png "Calculator")

Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


In [9]:
SELECT * FROM dates

,company_id,date_joined,year_founded
0,189,2017-06-24 00:00:00+00:00,1919
1,848,2021-06-01 00:00:00+00:00,2019
2,556,2022-02-15 00:00:00+00:00,2011
3,999,2021-11-17 00:00:00+00:00,2020
4,396,2021-10-21 00:00:00+00:00,2021
...,...,...,...
1069,1002,2019-04-29 00:00:00+00:00,2015
1070,215,2021-06-08 00:00:00+00:00,2003
1071,866,2021-09-21 00:00:00+00:00,2019
1072,651,2021-07-12 00:00:00+00:00,2020


In [10]:
SELECT * FROM funding

,company_id,valuation,funding,select_investors
0,189,4000000000,0,EQT Partners
1,848,1000000000,100000000,"Dragonfly Captial, Qiming Venture Partners, DS..."
2,556,2000000000,100000000,"Blackstone, Bessemer Venture Partners"
3,999,1000000000,100000000,"Goldman Sachs Asset Management, 3L"
4,396,2000000000,100000000,"Insight Partners, Softbank Group, Connect Vent..."
...,...,...,...,...
1069,1002,1000000000,0,"DST Global, Sequoia Capital China, Gaorong Cap..."
1070,215,4000000000,0,Sequoia Capital
1071,866,1000000000,0,"Advent International, PSG, Providence Equity P..."
1072,651,1000000000,0,"NetEase Capital, Northern Light Venture Capita..."


In [31]:
select 
      industry,
	  year_founded as year, 
	  count(*) as num_unicorns
from industries as i
inner join dates as d
on i.company_id = d.company_id 
where year_founded in (2019,2020,2021)
group by industry, year
order by industry

,industry,year,num_unicorns
0,Artificial intelligence,2019,5
1,Auto & transportation,2019,1
2,Consumer & retail,2020,2
3,Cybersecurity,2019,4
4,Cybersecurity,2020,4
5,Data management & analytics,2019,2
6,Data management & analytics,2021,1
7,E-commerce & direct-to-consumer,2019,4
8,E-commerce & direct-to-consumer,2020,4
9,E-commerce & direct-to-consumer,2021,3


In [4]:
select 
    industry,       
	EXTRACT(year FROM d.date_joined) AS year,
    count(industries.company_id) as num_unicorns,
	round(avg(valuation)/1000000000,2) as average_valuation_billions
	
from industries 	
inner join funding on
industries.company_id = funding.company_id 
inner join dates d
on industries.company_id = d.company_id
where 
       EXTRACT(year FROM d.date_joined) in ('2019', '2020', '2021') and industry in ('Fintech','Internet software & services','E-commerce & direct-to-consumer')

group by industry, year
order by industry, year desc;

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23


In [14]:
select industry,count(*) as num from industries
inner join dates
on industries.company_id = dates.company_id
where year_founded in (2021,2020,2019)
group by industry
order by num desc
limit 3

,industry,num
0,Fintech,18
1,Internet software & services,14
2,E-commerce & direct-to-consumer,11


In [1]:
with top_industries(industry) as
(select 
 i.industry,
 count(i.*) as num_unicorns 
 from industries as i
inner join dates d
on i.company_id = d.company_id
where EXTRACT(year FROM d.date_joined) in ('2019', '2020', '2021')
group by industry
order by num_unicorns desc
limit 3
),

yearly_rankings(industry,year,num_unicorns,average_valuation_billions) as
(
	select i.industry,
EXTRACT(year FROM d.date_joined) AS year,
 count(i.*) as num_unicorns,
 round(avg(f.valuation)/1000000000,2) as average_valuation_billions
from industries as i
inner join dates as d
on i.company_id = d.company_id
inner join funding as f
on d.company_id = f.company_id
group by industry,year
)

select industry,
    year,
	num_unicorns,
	average_valuation_billions
from yearly_rankings
where year in ('2019','2020','2021') and
industry in (
	select industry 
			 from top_industries)
group by industry, year, num_unicorns, average_valuation_billions
order by industry, year desc;


,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23
